In [1]:
import os
import re
import tabula
import pandas as pd
import numpy as np

In [2]:
# Set the JAVA_HOME environment variable to the Java installation directory
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk"

In [3]:
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
%pip install -q tabula-py
%pip install requests beautifulsoup4
%pip install openpyxl
%pip install selenium
%pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests

# Folder to store the downloaded files
download_folder = "pdf_downloads"
os.makedirs(download_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Function to extract data from the dynamically loaded HTML
def extract_data(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    driver.get(url)
    driver.implicitly_wait(10)  # Wait for elements to load

    data = []

    for row in rows:
        try:
            # Find the date in the same row
            date_element = row.find_element(By.XPATH, ".//td[contains(@class, 'views-field-field-date-created')]")
            date_text = date_element.text.strip()

        for row in rows[1:]:  # Skip header row
            cells = row.find_elements(By.TAG_NAME, 'td')
            if len(cells) >= 4:
                month = cells[0].text.strip()
                year = cells[1].text.strip()
                type_ = cells[2].text.strip()
                view_link_element = cells[3].find_element(By.TAG_NAME, 'a')
                view_link = view_link_element.get_attribute('href').strip()

                # Debugging output
                print(f"Month: {month}, Year: {year}, Type: {type_}, View Link: {view_link}")

                data.append({
                    "month": month,
                    "year": year,
                    "type": type_,
                    "view_link": view_link
                })
    except Exception as e:
        print(f"An error occurred while extracting data: {e}")

    driver.quit()
    return data

# Function to download a file
def download_file(url, file_name):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        file_path = os.path.join(download_folder, file_name)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"Downloaded: {file_name}")
    except requests.RequestException as e:
        print(f"Failed to download {file_name}: {e}")

# Main
website_url = "https://www.pelco3.org/rates.php"

# Extract data and download files
data_list = extract_data(website_url)
for item in data_list:
    # Ensure the file name has a valid PDF extension
    file_name = f"{item['year']}_{item['month']}_{item['type']}.pdf"
    download_file(item['view_link'], file_name)
